In [1]:
%load_ext autoreload


In [2]:
%autoreload 2

In [3]:
import pandas as pd
import numpy as np
from scipy import stats
from datetime import datetime
''' Sn properties - Masses, Ratios, Cup Configurations, ... '''
from sn_config import *
'''Classes for Reading in the Data and applying Internal Normalisation'''
from nu_data_reduction import NU_data_read, int_norm, evaluation
from pylab import plot, show, savefig
from scipy import stats
import os.path
import re
#%pylab inline

In [10]:
path = "/Volumes/friebelm/PhD/NU Plasma/Measurements/2015-03-25/"
path_sav = "/Volumes/friebelm/PhD/NU Plasma/Measurements/compilation/"
path_all = "/Volumes/friebelm/PhD/NU Plasma/Measurements/2015-09-28/"
files_start = 1987
files_end = 2173


In [13]:
# cup configuration
#cup_config = cycle_Sb
cup_config = cycles2
# Isotopes used for Interference correction
corr_isotopes_1 = {"Cd" : "111", "Te" : "126"}
corr_isotopes_2 = {"Cd" : "111","Te" : "125", "Xe" : "129"}
corr_isotopes_Sb = {"Te": "125"}
# Mass Range of cup configuration
mass_range = cycles2_mass_range
#isotopes = [["111", "112", "114", "115", "116", "117", "118", "119", "122", "124"], ["117", "118", "119", "122", "124"]]
isotopes = [["111", "112", "114", "115", "116", "117", "118", "119", "122", "124", "125", "126", "129"]]
#isotopes = [["116", "117", "118", "119", "122", "124"]]
denom_isotope = "120"
isotope_column = ["111", "112", "114", "115", "116", "117", "118", "119", "122", "124", "125", "126", "129"]
data_sample_column = [(i + "/" + "120") for i in isotope_column]
print data_sample_column
#number of iterations for beta
iter_beta = 10

# Interference_corr on the denominator isotope
isotope_denom_corr = True

blk_corr = True

['111/120', '112/120', '114/120', '115/120', '116/120', '117/120', '118/120', '119/120', '122/120', '124/120', '125/120', '126/120', '129/120']


In [14]:
filenames = os.listdir(path_all)

In [15]:
blk_ls = []
sample_ls = []
std_ls = []
for i in filenames:
    sample = re.search(r""+"Data_"+"(.*).csv", i)
    if sample == None:
        None
    else:
        sample = sample.group(1)
        df = NU_data_read(path_all, sample, cup_config)
        sample_name = df.extract_metadata(sample, "Sample Name")
        if sample_name == "blank sol" or sample_name == "wash" or sample_name == "wash clean":
            blk_ls.append(sample)
        elif sample_name == "SPEX 100ppb" or sample_name == "NIST 100ppb":
            std_ls.append(sample)
        else:
            sample_ls.append(sample)

In [16]:
columns = ["cycle", "sample", "date","H10 (1)", "H9 (1)", "H8 (1)", "H7 (1)", "H6 (1)", "H5 (1)", "H4 (1)", "H3 (1)", "H2 (1)", "H1 (1)", "Ax (1)", "L1 (1)", "L2 (1)", "L3 (1)", "L4 (1)", 
             "H10 (2)", "H9 (2)", "H8 (2)", "H7 (2)", "H6 (2)", "H5 (2)", "H4 (2)", "H3 (2)", "H2 (2)", "H1 (2)", "Ax (2)", "L1 (2)", "L2 (2)", "L3 (2)", "L4 (2)"]
df_all = pd.DataFrame(columns = columns)
for sample in blk_ls:
    df = NU_data_read(path_all, sample, cup_config)
    datafile = "Data_" + str(sample) + ".csv"
    data = pd.read_csv(path_all + datafile, skiprows=58, index_col='Cycle')
    if "H8 (2)" in data.columns:
        data_zero = data.ix[:,"H10 (Z1)":"L4 (Z1)"]
        data_1 = data.ix[:,"H10 (1)":"L4 (1)"]
        data_2 = data.ix[:,"H10 (2)":"L4 (2)"]
        data_zero.columns = data_1.columns
        data_1_corr = data_1 - data_zero
        data_1_corr["cycle"] = data_1_corr.index.values
        data_zero.columns = data_2.columns
        data_2_corr = data_2 - data_zero
        data_2_corr["cycle"] = data_2_corr.index.values
        data_1_corr = data_1_corr.merge(data_2_corr)
    else:
        data_zero = data.ix[:,"H10 (Z1)":"L4 (Z1)"]
        data_1 = data.ix[:,"H10 (1)":"L4 (1)"]
        data_zero.columns = data_1.columns
        data_1_corr = data_1 - data_zero
        data_1_corr["cycle"] = data_1_corr.index.values
        
    sample_name = df.extract_metadata(sample, "Sample Name")
    date = df.extract_metadata(sample, "Date")
    starttime = df.extract_metadata(sample, "Start Time")
    data_1_corr["date"] = datetime.strptime(date+starttime, '%d/%m/%Y%H:%M')
    data_1_corr["sample"] = sample_name
    df_all = pd.concat([df_all,data_1_corr], axis=0, ignore_index=True)
    #df_all = df_all.append(data_1_corr)
    df_all = df_all[columns]
df_all.set_index("date", inplace = True)
df_all

,cycle,sample,H10 (1),H9 (1),H8 (1),H7 (1),H6 (1),H5 (1),H4 (1),H3 (1),...,H5 (2),H4 (2),H3 (2),H2 (2),H1 (2),Ax (2),L1 (2),L2 (2),L3 (2),L4 (2)
date,,,,,,,,,,,,,,,,,,,,,
2015-09-29 08:55:00,1,blank sol,0,0,0.000272,0.000198,0.003423,0.000330,0.000959,0.000284,...,0.000080,3.117750e-05,0.000299,0.000021,0.000192,0.000090,0.003603,0.000352,0.001062,0.000351
2015-09-29 08:55:00,2,blank sol,0,0,0.000370,0.000264,0.003836,0.000449,0.001267,0.000406,...,0.000035,1.956620e-05,0.000250,0.000018,0.000123,0.000078,0.003339,0.000341,0.000886,0.000269
2015-09-29 08:55:00,3,blank sol,0,0,0.000332,0.000248,0.003750,0.000391,0.001188,0.000393,...,0.000038,2.671020e-05,0.000240,0.000038,0.000177,0.000107,0.003343,0.000350,0.000885,0.000275
2015-09-29 08:55:00,4,blank sol,0,0,0.000253,0.000206,0.003457,0.000337,0.000958,0.000291,...,0.000098,4.635590e-05,0.000280,0.000069,0.000152,0.000070,0.003242,0.000287,0.000819,0.000243
2015-09-29 08:55:00,5,blank sol,0,0,0.000246,0.000176,0.003235,0.000278,0.000822,0.000255,...,0.000053,3.501900e-06,0.000260,0.000047,0.000188,0.000077,0.003393,0.000329,0.000915,0.000301
2015-09-29 08:55:00,6,blank sol,0,0,0.000275,0.000186,0.003411,0.000331,0.000940,0.000302,...,0.000062,2.849990e-05,0.000322,0.000025,0.000252,0.000075,0.003696,0.000432,0.001173,0.000384
2015-09-29 08:55:00,7,blank sol,0,0,0.000278,0.000184,0.003299,0.000313,0.000871,0.000264,...,0.000073,2.671510e-05,0.000233,0.000016,0.000140,0.000095,0.003295,0.000324,0.000897,0.000282
2015-09-29 08:55:00,8,blank sol,0,0,0.000305,0.000245,0.003581,0.000364,0.001083,0.000330,...,0.000049,2.224810e-05,0.000314,0.000025,0.000288,0.000081,0.003845,0.000432,0.001262,0.000400
2015-09-29 08:55:00,9,blank sol,0,0,0.000259,0.000158,0.003345,0.000308,0.000896,0.000303,...,0.000060,1.867920e-05,0.000280,0.000048,0.000262,0.000045,0.003541,0.000373,0.001062,0.000344


In [17]:
def mean(x):
    mean = np.nanmean(x) 
    return mean

df_all_mean = df_all.drop("cycle", axis=1).groupby(df_all.index).agg(mean)
df_all_mean["sample"] = df_all[df_all["cycle"] == 1]["sample"]
df_all_mean

,H10 (1),H9 (1),H8 (1),H7 (1),H6 (1),H5 (1),H4 (1),H3 (1),H2 (1),H1 (1),...,H4 (2),H3 (2),H2 (2),H1 (2),Ax (2),L1 (2),L2 (2),L3 (2),L4 (2),sample
date,,,,,,,,,,,,,,,,,,,,,
2015-09-28 16:54:00,0,0,0.000369,0.000253,0.003571,0.000459,0.001281,0.000411,0.000782,0.000053,...,2.791289e-05,0.000340,0.000051,0.000238,0.000084,0.003314,0.000391,0.001089,0.000354,blank sol
2015-09-28 17:12:00,0,0,0.000129,0.000078,0.001677,0.000113,0.000473,0.000125,0.000221,0.000026,...,2.264336e-05,0.000169,0.000044,0.000088,0.000056,0.001675,0.000174,0.000388,0.000047,blank sol
2015-09-28 18:36:00,0,0,0.000538,0.000386,0.004160,0.000688,0.001929,0.000624,0.001158,0.000041,...,1.170323e-05,0.000560,0.000035,0.000402,0.000056,0.004282,0.000736,0.002028,0.000635,blank sol
2015-09-28 20:48:00,0,0,0.000306,0.000199,0.004545,0.000365,0.000982,0.000324,0.000610,0.000034,...,1.532955e-05,0.000356,0.000083,0.000233,0.000147,0.004735,0.000410,0.001120,0.000361,blank sol
2015-09-28 21:33:00,0,0,0.000200,0.000119,0.003664,0.000211,0.000556,0.000177,0.000359,0.000026,...,1.100433e-05,0.000205,0.000058,0.000129,0.000082,0.003759,0.000226,0.000628,0.000197,blank sol
2015-09-28 21:55:00,0,0,0.000261,0.000163,0.003896,0.000301,0.000816,0.000259,0.000525,0.000024,...,1.366495e-05,0.000248,0.000072,0.000154,0.000084,0.003893,0.000287,0.000811,0.000253,blank sol
2015-09-28 22:18:00,0,0,0.000309,0.000198,0.004125,0.000383,0.001018,0.000307,0.000652,0.000036,...,5.865060e-06,0.000255,0.000068,0.000180,0.000074,0.003899,0.000296,0.000855,0.000274,blank sol
2015-09-28 22:41:00,0,0,0.000296,0.000201,0.004075,0.000364,0.000989,0.000318,0.000624,0.000041,...,2.100355e-06,0.000303,0.000056,0.000209,0.000079,0.004032,0.000349,0.000959,0.000296,blank sol
2015-09-28 23:04:00,0,0,0.000236,0.000161,0.003700,0.000278,0.000778,0.000249,0.000494,0.000041,...,9.410305e-06,0.000264,0.000046,0.000189,0.000094,0.003765,0.000302,0.000835,0.000260,blank sol


In [18]:
df_all_mean.to_csv(path_sav + "blanks_mean.csv", mode='a', header=False)

In [215]:
df_internal_sub.to_csv(path_sav + "Sn_H8_L4_2_cycles_internal_norm.csv", mode='a', header=False)

In [229]:
df_internal_norm_mean_sd.to_csv(path + "Sn_H8_L4_2_cycles_internal_norm_mean_no_bgd_corr.csv")

In [389]:
epsilon.to_csv(path + "Sn_H8_L4_2_cycles_internal_norm_epsilon_bgd_corr.csv")